In [1]:
import requests
import nacl.signing
import base64
import os
import hashlib
import random
import BaseHTTPServer
import socket

In [2]:
eduvpn_key = 'E5On0JTtyUVZmcWd+I/FXRm32nSq8R2ioyW7dcu/U88='

In [3]:
def csrf_token(nbytes=32):
    tok = os.urandom(nbytes)
    return base64.urlsafe_b64encode(tok).rstrip(b'=').decode('ascii')

def code_challenge(length=128):
    choices = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-._~'
    code_verifier = "".join(random.choice(choices) for _ in range(length))
    return base64.urlsafe_b64encode(hashlib.sha256(code_verifier).digest()).rstrip('=')

def get_open_port():
        """find an unused port"""
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(("",0))
        s.listen(1)
        port = s.getsockname()[1]
        s.close()
        return port

In [4]:
# retrieve a list of instances
inst_doc_url = 'https://static.eduvpn.nl/instances.json'
inst_doc_sig_url = 'https://static.eduvpn.nl/instances.json.sig'
inst_doc = requests.get(inst_doc_url)
inst_doc_sig = requests.get(inst_doc_sig_url)
instances = [i['base_uri'] for i in inst_doc.json()['instances']]
instances

[u'https://nl.eduvpn.org/',
 u'https://esciencecenter.eduvpn.nl/',
 u'https://ru.eduvpn.nl/',
 u'https://surf.eduvpn.nl/',
 u'https://ut.eduvpn.nl/',
 u'https://demo.eduvpn.nl/']

In [5]:
# verify signature
verify_key = nacl.signing.VerifyKey(eduvpn_key, encoder=nacl.encoding.Base64Encoder)
_ = verify_key.verify(smessage=inst_doc.content, signature=inst_doc_sig.content.decode('base64'))

In [6]:
instance = 'https://demo.eduvpn.nl/'

In [7]:
# get info from server
info = requests.get(instance + '/info.json').json()
urls = info['api']['http://eduvpn.org/api#2']
authorization_endpoint = urls['authorization_endpoint']

In [8]:
# construct authorization url
port = get_open_port()
params = {
    "client_id": "org.eduvpn.app",   #  the ID that was registered, see below;
    "redirect_uri":   'http://127.0.0.1:%s/callback' % port,   # the URL that was registered, see below;
    "response_type": "code",  # always code;
    "scope": "config",  # this is always config;
    "state": csrf_token(),  # a cryptographically secure random string, to avoid CSRF;
    "code_challenge_method": "S256",  # always S256;
    "code_challenge": code_challenge() # the code challenge (see RFC 7636).
}
url = authorization_endpoint + "?" + "&".join([k + "=" + v for k, v in params.items()])

In [9]:
# open authorization url
print(url)
!open "$url"

In [10]:
# listen for one request
class RequestHandler(BaseHTTPServer.BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write("<html><head><title>eduvpn</title></head><body><h1>You can now close this window</h1>")
        self.server.path = self.path

httpd = BaseHTTPServer.HTTPServer(('', port), RequestHandler)
httpd.handle_request()

127.0.0.1 - - [13/Jul/2017 14:49:18] "GET /callback?code=KwiU%2Bb90J9iFWMuJ5CNn12bDsDBXz92XrY%2B0srsUcFvTeeLVgX2zME7XsLCoMqyqE712CiTEWxC%2F%2FQSMOK53BHsidHlwZSI6ImF1dGhvcml6YXRpb25fY29kZSIsImF1dGhfa2V5IjoiYWNiMzI5MWNmZGU2NjAzZDNjMzgxNzBjOGE4MDY1YTUiLCJ1c2VyX2lkIjoiZTExMzdlZjc0MmFkZjgzMDhhYWI3ZTBhNmI2NDNjOGUyMmNiZjNmYiIsImNsaWVudF9pZCI6Im9yZy5lZHV2cG4uYXBwIiwic2NvcGUiOiJjb25maWciLCJyZWRpcmVjdF91cmkiOiJodHRwOlwvXC8xMjcuMC4wLjE6NTA3NTFcL2NhbGxiYWNrIiwiY29kZV9jaGFsbGVuZ2UiOiJBYkRsUGlQcUtOb3RPU3ZLaFdoMDV6ZzVpYVI2WW5YLU1ObXdYX3RLQWJRIiwiZXhwaXJlc19hdCI6IjIwMTctMDctMTMgMTE6NTQ6MTgifQ%3D%3D&state=fZH5tb4zyPhtAb2cAvYw0wszvTr9zqL7uwp5rhm9hjA HTTP/1.1" 200 -


In [11]:
# parse the path
path, args_raw = httpd.path.split("?")
args = dict([i.split('=') for i in args_raw.split("&")])
token = args['code']
token

'KwiU%2Bb90J9iFWMuJ5CNn12bDsDBXz92XrY%2B0srsUcFvTeeLVgX2zME7XsLCoMqyqE712CiTEWxC%2F%2FQSMOK53BHsidHlwZSI6ImF1dGhvcml6YXRpb25fY29kZSIsImF1dGhfa2V5IjoiYWNiMzI5MWNmZGU2NjAzZDNjMzgxNzBjOGE4MDY1YTUiLCJ1c2VyX2lkIjoiZTExMzdlZjc0MmFkZjgzMDhhYWI3ZTBhNmI2NDNjOGUyMmNiZjNmYiIsImNsaWVudF9pZCI6Im9yZy5lZHV2cG4uYXBwIiwic2NvcGUiOiJjb25maWciLCJyZWRpcmVjdF91cmkiOiJodHRwOlwvXC8xMjcuMC4wLjE6NTA3NTFcL2NhbGxiYWNrIiwiY29kZV9jaGFsbGVuZ2UiOiJBYkRsUGlQcUtOb3RPU3ZLaFdoMDV6ZzVpYVI2WW5YLU1ObXdYX3RLQWJRIiwiZXhwaXJlc19hdCI6IjIwMTctMDctMTMgMTE6NTQ6MTgifQ%3D%3D'